<a href="https://colab.research.google.com/github/KacperKaczmarczyk/data-science-bootcamp/blob/main/06_uczenie_glebokie/02_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Tensorflow
>Strona biblioteki: [https://www.tensorflow.org/](https://www.tensorflow.org/)  
>Dokumentacja: [https://www.tensorflow.org/api_docs](https://www.tensorflow.org/api_docs)  
>
>Szybko rozwijająca się biblioteka do uczenia maszynowego i uczenia głębokiego rozwijana przez Google.
>
>Aby zainstalować bibliotekę tensorflow, użyj polecenia poniżej:
```
pip install tensorflow
```

### Keras
>Strona biblioteki: [https://keras.io/](https://keras.io/)  
>
>Wysokopoziomowy interfejs do budowy sieci neuronowych. Działa zarówno na Tensorflow, Theano oraz CNTK.
>
>Aby zainstalować bibliotekę keras, użyj polecenia poniżej:
```
pip install keras
```

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Załadowanie biblioteki tensorflow


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


'2.11.0'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [4]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [5]:
from tensorflow.keras.layers import Dense


model.add(Dense(units=4, input_shape=(10,)))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________


Dodanie kolejnej warstwy

In [7]:
model.add(Dense(units=2))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
 dense_1 (Dense)             (None, 2)                 10        
                                                                 
Total params: 54
Trainable params: 54
Non-trainable params: 0
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiazywać problemy nieliniowe.

[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [8]:
from tensorflow.keras.activations import linear

random_data = np.linspace(start=-3, stop=3, num=300)
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-3.000000,-3.000000
1,-2.979933,-2.979933
2,-2.959866,-2.959866
3,-2.939799,-2.939799
4,-2.919732,-2.919732


In [9]:
px.line(data, x='data', y='linear', width=500, height=400, range_y=[-3, 3])

In [10]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-3.000000,0.047426
1,-2.979933,0.048341
2,-2.959866,0.049272
3,-2.939799,0.050221
4,-2.919732,0.051187


In [11]:
px.line(data, x='data', y='sigmoid', width=500, height=400, range_y=[-0.5, 1.5])

In [12]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-3.000000,0.0
1,-2.979933,0.0
2,-2.959866,0.0
3,-2.939799,0.0
4,-2.919732,0.0


In [13]:
px.line(data, x='data', y='relu', width=500, height=400, range_y=[-0.5, 1.5])

In [14]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-3.000000,-0.995055
1,-2.979933,-0.994853
2,-2.959866,-0.994643
3,-2.939799,-0.994424
4,-2.919732,-0.994196


In [15]:
px.line(data, x='data', y='tanh', width=500, height=400, range_y=[-1.5, 1.5])

In [16]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/)) 
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [17]:
# klasyfikacja binarna
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [18]:
# model.fit(data, labels, epochs=10, batch_size=32)
# model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epochs=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [19]:
data = np.random.randn(10000, 150)
labels = np.random.randint(2, size=(10000, 1))

print(data.shape)
print(labels.shape)

(10000, 150)
(10000, 1)


In [20]:
data[:3]

array([[ 2.54632215e+00,  4.49356502e-01, -8.27210066e-01,
         1.43667639e-01,  8.17913292e-02, -4.16435326e-01,
         2.27153928e+00,  1.24188714e+00, -1.07267969e+00,
         8.30058710e-01, -6.23050158e-01, -1.20141291e+00,
        -8.73455366e-01, -3.25171892e-01, -1.88781059e+00,
         1.35948982e+00,  4.04782369e-01,  2.90768204e-01,
         1.17755334e-01,  1.16620293e+00, -1.52258493e+00,
        -9.64189906e-01, -8.55254819e-01, -7.55653622e-01,
        -8.30641933e-01,  8.95165110e-01,  2.52636153e-01,
         5.06232494e-01,  1.29783318e-01, -9.54901160e-01,
        -5.03497625e-01, -1.17051823e+00,  1.10520938e-01,
        -1.21787683e+00,  3.10892216e+00,  3.26444734e-02,
        -1.23934491e+00,  6.72149673e-01,  3.66350900e-01,
         9.18928213e-02,  6.76878617e-01,  2.82517520e-01,
         1.83328447e-01, -5.79786774e-01, -5.83551691e-01,
         7.90209826e-01,  6.52231570e-01,  4.41775124e-01,
        -3.95602402e-01, -5.35627543e-01, -8.85609301e-0

In [21]:
labels[:10]

array([[1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0]])

In [22]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Epoch 1/20
313/313 [==============================] - 1s 2ms/step - loss: 0.7745 - accuracy: 0.4949
Epoch 2/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6944 - accuracy: 0.5380
Epoch 3/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6759 - accuracy: 0.5705
Epoch 4/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6626 - accuracy: 0.6009
Epoch 5/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6493 - accuracy: 0.6257
Epoch 6/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6361 - accuracy: 0.6409
Epoch 7/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6216 - accuracy: 0.6594
Epoch 8/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6072 - accuracy: 0.6782
Epoch 9/20
313/313 [==============================] - 1s 2ms/step - loss: 0.5929 - accuracy: 0.6897
Epoch 10/20
313/313 [==============================] - 1s 2ms/step - loss: 0.5794 - accuracy: 0.7053

In [23]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=64)

Epoch 1/20
157/157 [==============================] - 1s 2ms/step - loss: 0.7458 - accuracy: 0.5047
Epoch 2/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6975 - accuracy: 0.5420
Epoch 3/20
157/157 [==============================] - 0s 3ms/step - loss: 0.6817 - accuracy: 0.5652
Epoch 4/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6705 - accuracy: 0.5873
Epoch 5/20
157/157 [==============================] - 1s 6ms/step - loss: 0.6606 - accuracy: 0.6054
Epoch 6/20
157/157 [==============================] - 0s 3ms/step - loss: 0.6509 - accuracy: 0.6256
Epoch 7/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6417 - accuracy: 0.6390
Epoch 8/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6321 - accuracy: 0.6527
Epoch 9/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6225 - accuracy: 0.6626
Epoch 10/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6124 - accuracy: 0.6782

In [24]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
250/250 [==============================] - 1s 4ms/step - loss: 0.7572 - accuracy: 0.4991 - val_loss: 0.7164 - val_accuracy: 0.5185
Epoch 2/20
250/250 [==============================] - 1s 3ms/step - loss: 0.6961 - accuracy: 0.5476 - val_loss: 0.7140 - val_accuracy: 0.5195
Epoch 3/20
250/250 [==============================] - 1s 3ms/step - loss: 0.6744 - accuracy: 0.5814 - val_loss: 0.7147 - val_accuracy: 0.5155
Epoch 4/20
250/250 [==============================] - 1s 3ms/step - loss: 0.6580 - accuracy: 0.6068 - val_loss: 0.7162 - val_accuracy: 0.5140
Epoch 5/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6420 - accuracy: 0.6300 - val_loss: 0.7208 - val_accuracy: 0.5115
Epoch 6/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6268 - accuracy: 0.6524 - val_loss: 0.7263 - val_accuracy: 0.5275
Epoch 7/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6115 - accuracy: 0.6715 - val_loss: 0.7344 - val_accuracy: 0.5135
Epoch 

In [25]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [26]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [27]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/20
250/250 [==============================] - 1s 3ms/step - loss: 0.7630 - accuracy: 0.5033 - val_loss: 0.7192 - val_accuracy: 0.5150
Epoch 2/20
250/250 [==============================] - 1s 3ms/step - loss: 0.6981 - accuracy: 0.5412 - val_loss: 0.7119 - val_accuracy: 0.5245
Epoch 3/20
250/250 [==============================] - 1s 3ms/step - loss: 0.6761 - accuracy: 0.5719 - val_loss: 0.7114 - val_accuracy: 0.5335
Epoch 4/20
250/250 [==============================] - 1s 3ms/step - loss: 0.6586 - accuracy: 0.6065 - val_loss: 0.7118 - val_accuracy: 0.5315
Epoch 5/20
250/250 [==============================] - 1s 3ms/step - loss: 0.6428 - accuracy: 0.6335 - val_loss: 0.7165 - val_accuracy: 0.5165
Epoch 6/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6280 - accuracy: 0.6544 - val_loss: 0.7207 - val_accuracy: 0.5170
Epoch 7/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6116 - accuracy: 0.6715 - val_loss: 0.7262 - val_accuracy: 0.5305
Epoch 

In [28]:
test_data = np.random.randn(5, 150)
test_labels = np.random.randint(2, size=(5, 1))

In [29]:
model.predict(test_data)

1/1 [==============================] - 0s 84ms/step


array([[0.20019563],
       [0.3078052 ],
       [0.09953022],
       [0.63458204],
       [0.38955566]], dtype=float32)

AttributeError: ignored

In [32]:
np.argmax(model.predict(test_data),axis=1)

1/1 [==============================] - 0s 48ms/step


array([0, 0, 0, 0, 0])

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [34]:
data = np.random.random((10000, 150))
labels = np.random.randint(10, size=(10000, 1))

In [35]:
print(data.shape)
print(labels.shape)

(10000, 150)
(10000, 1)


In [36]:
labels[:10]

array([[0],
       [8],
       [6],
       [8],
       [9],
       [4],
       [4],
       [8],
       [1],
       [0]])

In [37]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=10)
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [38]:
labels[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [39]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
250/250 [==============================] - 1s 3ms/step - loss: 2.3171 - accuracy: 0.1077 - val_loss: 2.3059 - val_accuracy: 0.0980
Epoch 2/30
250/250 [==============================] - 1s 2ms/step - loss: 2.3040 - accuracy: 0.1001 - val_loss: 2.3063 - val_accuracy: 0.1030
Epoch 3/30
250/250 [==============================] - 1s 2ms/step - loss: 2.3018 - accuracy: 0.1064 - val_loss: 2.3037 - val_accuracy: 0.0970
Epoch 4/30
250/250 [==============================] - 0s 2ms/step - loss: 2.3011 - accuracy: 0.1063 - val_loss: 2.3056 - val_accuracy: 0.0920
Epoch 5/30
250/250 [==============================] - 1s 2ms/step - loss: 2.2999 - accuracy: 0.1106 - val_loss: 2.3106 - val_accuracy: 0.0995
Epoch 6/30
250/250 [==============================] - 1s 2ms/step - loss: 2.2989 - accuracy: 0.1123 - val_loss: 2.3101 - val_accuracy: 0.0920
Epoch 7/30
250/250 [==============================] - 1s 2ms/step - loss: 2.2968 - accuracy: 0.1136 - val_loss: 2.3122 - val_accuracy: 0.1030
Epoch 

In [40]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 57ms/step


array([[0.10068253, 0.06644502, 0.10025642, 0.12065302, 0.10590167,
        0.0652381 , 0.149693  , 0.0614068 , 0.13689654, 0.09282692],
       [0.13026783, 0.09212829, 0.09147646, 0.04979193, 0.05254286,
        0.1395332 , 0.12684844, 0.1332347 , 0.14067316, 0.0435032 ],
       [0.13516977, 0.06522663, 0.06469104, 0.19191156, 0.21863066,
        0.10308824, 0.03987984, 0.04601322, 0.06799055, 0.06739835],
       [0.07142453, 0.11452693, 0.06092588, 0.09470538, 0.13318378,
        0.10340764, 0.13298137, 0.10201403, 0.08717065, 0.09965977],
       [0.14326586, 0.09541529, 0.09269822, 0.0828671 , 0.07342821,
        0.10535616, 0.11106849, 0.12683257, 0.10565943, 0.06340867],
       [0.0621366 , 0.08433206, 0.12587602, 0.08078048, 0.13073258,
        0.11046509, 0.06401356, 0.11490382, 0.102143  , 0.12461684],
       [0.11336462, 0.18904467, 0.05737818, 0.06973871, 0.0821958 ,
        0.07079789, 0.13370125, 0.12597433, 0.07468363, 0.08312085],
       [0.03897227, 0.16611531, 0.0731357

In [41]:
np.argmax(model.predict(test_data),axis=1)

1/1 [==============================] - 0s 99ms/step


array([6, 8, 4, 4, 0, 4, 1, 1, 0, 0])

### <a name='a6'></a> Przykład - regresja

In [42]:
data = np.random.random((10000, 150))
labels = 50 * np.random.random(10000)

In [43]:
labels[:10]

array([26.72534755, 11.95440505, 27.10921212, 39.1772944 ,  5.87110242,
       16.00772824, 44.65630732, 36.28820959, 12.73133663, 10.75803384])

In [44]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
250/250 [==============================] - 1s 2ms/step - loss: 248.6677 - val_loss: 204.6831
Epoch 2/30
250/250 [==============================] - 1s 2ms/step - loss: 207.2362 - val_loss: 205.0195
Epoch 3/30
250/250 [==============================] - 1s 2ms/step - loss: 206.9458 - val_loss: 204.6856
Epoch 4/30
250/250 [==============================] - 0s 2ms/step - loss: 206.4743 - val_loss: 204.8032
Epoch 5/30
250/250 [==============================] - 0s 2ms/step - loss: 206.3003 - val_loss: 206.2145
Epoch 6/30
250/250 [==============================] - 0s 2ms/step - loss: 205.9499 - val_loss: 204.7340
Epoch 7/30
250/250 [==============================] - 1s 2ms/step - loss: 205.4724 - val_loss: 204.7088
Epoch 8/30
250/250 [==============================] - 0s 2ms/step - loss: 205.7194 - val_loss: 204.7800
Epoch 9/30
250/250 [==============================] - 0s 2ms/step - loss: 205.5314 - val_loss: 204.8215
Epoch 10/30
250/250 [==============================] - 0s 2ms/st

In [45]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 1s 3ms/step - loss: 13.8186 - mse: 273.6679 - val_loss: 12.3145 - val_mse: 204.7736
Epoch 2/10
250/250 [==============================] - 1s 2ms/step - loss: 12.4755 - mse: 207.5775 - val_loss: 12.3574 - val_mse: 205.8832
Epoch 3/10
250/250 [==============================] - 1s 2ms/step - loss: 12.4542 - mse: 207.2345 - val_loss: 12.3461 - val_mse: 206.4607
Epoch 4/10
250/250 [==============================] - 1s 2ms/step - loss: 12.4448 - mse: 206.9578 - val_loss: 12.3427 - val_mse: 205.1632
Epoch 5/10
250/250 [==============================] - 1s 2ms/step - loss: 12.4254 - mse: 206.5968 - val_loss: 12.3715 - val_mse: 206.2682
Epoch 6/10
250/250 [==============================] - 0s 2ms/step - loss: 12.4310 - mse: 206.7830 - val_loss: 12.3220 - val_mse: 204.5684
Epoch 7/10
250/250 [==============================] - 0s 2ms/step - loss: 12.4135 - mse: 206.2895 - val_loss: 12.3317 - val_mse: 204.8429
Epoch 8/10
250/250 [==============

In [46]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 60ms/step


array([[24.817833],
       [27.089125],
       [29.457605],
       [28.586235],
       [26.56279 ],
       [28.732697],
       [26.740332],
       [28.576511],
       [24.623535],
       [23.497921]], dtype=float32)